In [1]:
import gym
import pybullet
import pybullet_envs
from gym import wrappers
from datetime import datetime

# DR TRPO related files
from train_helper import *
from value import NNValueFunction
from utils import Logger
from dr_policy import DRPolicyKL, DRPolicyWass

In [2]:
#########################       Discrete State Space - KL DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'Taxi-v3'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 2500
batch_eps = 100
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        policy.update(observes, actions, advantages)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 3, h2: 3, h3: 5, lr: 0.00577
***** Episode 100, Mean Return = -766.6, Mean Discounted Return = -19.4 *****
ExplainedVarNew: -5.93e-10
ExplainedVarOld: -9.52e-07
ValFuncLoss: 56


***** Episode 200, Mean Return = -420.2, Mean Discounted Return = -11.1 *****
ExplainedVarNew: -0.000792
ExplainedVarOld: -7.66e-10
ValFuncLoss: 48.1


***** Episode 300, Mean Return = -231.9, Mean Discounted Return = -6.6 *****
ExplainedVarNew: -0.00716
ExplainedVarOld: -0.00369
ValFuncLoss: 16.8


***** Episode 400, Mean Return = -177.2, Mean Discounted Return = -5.4 *****
ExplainedVarNew: -0.000432
ExplainedVarOld: -0.0118
ValFuncLoss: 7.72


***** Episode 500, Mean Return = -159.2, Mean Discounted Return = -5.2 *****
ExplainedVarNew: -0.000127
ExplainedVarOld: -0.000412
ValFuncLoss: 7.05


***** Episode 600, Mean Return = -124.2, Mean Discounted Return = -5.1 *****
ExplainedVarNew: -0.000176
ExplainedVarOld: -7.65e-05
ValFuncLoss: 9.5


***** Episode 700, Mean Return = -76.0, Mean Disco

In [ ]:
#########################       Discrete State Space - Wasserstein DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'Taxi-v3'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 5000
batch_eps = 100
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        policy.update(observes, actions, advantages, disc_freqs)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()